In [110]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise import cosine_similarity


In [111]:
# goodreads_df = pd.read_csv("../Resources/goodreads_dataset.csv")
goodreads_df= pd.read_csv("../Resources/goodreads_dataset.csv") 


In [112]:
reviews_df = pd.read_csv("../Resources/book_reviews.csv")


In [78]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17158 entries, 0 to 17157
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  17158 non-null  object 
 1   Title               17158 non-null  object 
 2   Price               10897 non-null  float64
 3   User_id             14573 non-null  object 
 4   profileName         14572 non-null  object 
 5   review/helpfulness  17158 non-null  object 
 6   review/score        17158 non-null  float64
 7   review/time         17158 non-null  int64  
 8   review/summary      17155 non-null  object 
 9   review/text         17158 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.3+ MB


In [79]:
reviews_df.isna().sum()

Id                       0
Title                    0
Price                 6261
User_id               2585
profileName           2586
review/helpfulness       0
review/score             0
review/time              0
review/summary           3
review/text              0
dtype: int64

In [80]:
goodreads_df.isna().sum()

title                 0
titleComplete         0
description           0
genres                0
isbn               2974
publisher           670
author                2
characters         7535
places             8741
ratingHistogram       3
ratingsCount          3
reviewsCount         20
numPages            245
language            354
dtype: int64

In [113]:
import ast
# Function to safely parse string lists
def parse_string_list(column):
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else [])

In [114]:
# Convert 'numPages' to numeric first (in case there are non-numeric values)
goodreads_df['numPages'] = pd.to_numeric(goodreads_df['numPages'], errors='coerce')

# Handle missing values
goodreads_df = goodreads_df.assign(
    isbn=goodreads_df['isbn'].fillna('Unknown'),
    publisher=goodreads_df['publisher'].fillna('Unknown'),
    author=goodreads_df['author'].fillna('Unknown'),
    numPages=goodreads_df['numPages'].fillna(goodreads_df['numPages'].median()),  # ✅ Corrected
    language=goodreads_df['language'].fillna('Unknown')
)


In [116]:
# Convert numerical fields to integers
goodreads_df.dropna(subset=['ratingsCount', 'reviewsCount'], inplace=True)
goodreads_df['ratingsCount'] = goodreads_df['ratingsCount'].astype(int)
goodreads_df['reviewsCount'] = goodreads_df['reviewsCount'].astype(int)
goodreads_df['numPages'] = goodreads_df['numPages'].astype(int)

In [117]:
# Cleaning Book Reviews dataset
reviews_df = reviews_df.assign(
    Price=reviews_df['Price'].fillna(reviews_df['Price'].median()),
    User_id=reviews_df['User_id'].fillna('Unknown'),
    profileName=reviews_df['profileName'].fillna('Unknown')
)

In [118]:
#  Ensure that "1/5" becomes 1.0, "2/5" becomes 2.0
reviews_df['review/helpfulness'] = reviews_df['review/helpfulness'].str.split('/').str[0].astype(float)


In [121]:
# Standardizing column names for merging
# Rename columns for consistency
goodreads_df.rename(columns={'title': 'Title'}, inplace=True)

# Merging the datasets on 'Title'
merged_df = pd.merge(reviews_df, goodreads_df, on="Title", how="inner")

# Drop duplicates after merging
merged_df.drop_duplicates(subset=['Title'], inplace=True)
# Handle missing values
merged_df.fillna({'author': 'Unknown', 'genres': 'Unknown', 'isbn': 'Unknown'}, inplace=True)

# Convert review time to datetime format
merged_df['review/time'] = pd.to_datetime(merged_df['review/time'], unit='s', errors='coerce')
merged_df


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,...,isbn,publisher,author,characters,places,ratingHistogram,ratingsCount,reviewsCount,numPages,language
0,1401201911,Superman: Red Son,11.42,A3BGNNL6VPLFRO,Michael Kleyn,1.0,5.0,2008-07-08,Amazing story,Great story that really draws you in. I am not...,...,1401201911,DC Comics,"['Kilian Plunkett', 'Mark Millar', 'Dave Johns...","['Lana Lang', 'Martha Clark Kent', 'Oliver Que...",NaN,"[826, 2095, 8872, 19070, 25172]",56035,2360,160,English
128,0553280589,Eva Luna,10.85,Unknown,Unknown,41.0,5.0,1997-02-12,The author's use of magic realism will spur yo...,Are you looking to add some vivid color into y...,...,0553280589,Bantam,['Isabel Allende'],"['El Negro', 'Rolf Carlé', 'Huberto Naranjo', ...",['Chile'],"[403, 1956, 11041, 20237, 15737]",49374,2086,307,English
216,0394751019,The Persian Boy,10.85,A25QX3XJJCZ23C,Hannah Lovette,4.0,5.0,2009-06-12,Truly Amazing (And this is coming from a reluc...,First off I want to say that I was not initial...,...,0394751019,Vintage,['Mary Renault'],"['Darius III', 'Roxana of Bactria', 'Hephaesti...","['Persia', 'Persian Empire']","[90, 302, 1497, 3301, 4102]",9292,705,420,English
489,0451204727,Black Cross,34.00,A3R1MVZG7ZZ295,"Sherrie A., Norman ""SherrieNorman""",1.0,5.0,2011-03-26,Black Cross,Black CrossI have always been most interested ...,...,0451204727,New American Library,['Greg Iles'],"['Jonas Stern', 'Mark McConnell', 'Anna Haas',...",NaN,"[107, 278, 1783, 4556, 5356]",12080,678,556,English
614,0816635277,The Good Woman of Setzuan,10.00,A3429NWUW5IYYY,Reader99,0.0,1.0,2011-01-23,Don't know-can't get what I ordered.,"Ordered ""The Good Woman of Setzuan by Brecht"" ...",...,0816635277,University of Minnesota Press,"['Eric Bentley', 'Bertolt Brecht']","['Shui Ta', 'Shen Te', 'Wang']",['China'],"[405, 1064, 2949, 3843, 2708]",10969,378,112,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9284,1896944329,Earthcore,10.85,A3LPFHKYNGKDHY,mario,4.0,4.0,2007-08-10,Great action book!,If you want to read a great action & adventure...,...,1896944329,Dragon Moon Press,['Scott Sigler'],NaN,NaN,"[101, 287, 1335, 2638, 2621]",6982,538,320,Unknown
9358,0679729895,Tales of the Unexpected,11.63,A2JBDC1OR1KZGP,Catherine S. Vodrey,29.0,5.0,2002-03-17,"Vintage Roald Dahl--wonderful, wonderful!",Roald Dahl is at the peak of his powers in thi...,...,0679729895,Random House Vintage Books,['Roald Dahl'],NaN,NaN,"[51, 256, 1596, 3940, 3980]",9823,533,471,English
9391,0156032899,Lighthousekeeping,16.95,A319KYEIAZ3SON,Mary Whipple,28.0,5.0,2005-04-14,"""My life is a hesitation in time, an opening i...",Jeanette Winterson's magnificently descriptive...,...,0156032899,Mariner Books,['Jeanette Winterson'],NaN,NaN,"[123, 594, 2242, 3317, 2618]",8894,872,232,English
9413,1591169151,"Ouran High School Host Club, Vol. 1",9.99,ACMK7RRUP9UEW,Haine,2.0,5.0,2006-11-09,Hillarious!,I enjoyed this book alot. Hatori-sensei has re...,...,1591169151,VIZ Media,['Bisco Hatori'],"['Tamaki Suou', 'Mitsukuni Haninozuka', 'Hikar...",NaN,"[3798, 5184, 19221, 38040, 106063]",172306,1731,178,English


In [124]:
# Ensure the 'review/helpfulness' column is of string type before splitting
merged_df['review/helpfulness'] = merged_df['review/helpfulness'].astype(str).str.split('/').str[0].astype(float).astype(int)
merged_df['review/score'] = merged_df['review/score'].astype(str).str.split('/').str[0].astype(float).astype(int)

In [93]:
import ast
import pandas as pd

# Ensure 'merged_df' is properly defined (simulate example)
# Here, we assume 'merged_df' is already loaded in the environment

# Check if 'ratingHistogram' column exists before processing
if 'ratingHistogram' in merged_df.columns:
    # Convert ratingHistogram from string to a list of numbers
    merged_df['ratingHistogram'] = merged_df['ratingHistogram'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    # Extract each rating count into its own column
    merged_df[['1_star', '2_star', '3_star', '4_star', '5_star']] = merged_df['ratingHistogram'].apply(
        lambda x: pd.Series(x) if isinstance(x, list) and len(x) == 5 else pd.Series([None]*5)
    )

    # Fill NaN values with 0 and convert to integers
    merged_df[['1_star', '2_star', '3_star', '4_star', '5_star']] = merged_df[['1_star', '2_star', '3_star', '4_star', '5_star']].fillna(0).astype(int)

# Display updated dataframe
merged_df.head()


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,...,ratingHistogram,ratingsCount,reviewsCount,numPages,language,1_star,2_star,3_star,4_star,5_star
0,1401201911,Superman: Red Son,11.42,A3BGNNL6VPLFRO,Michael Kleyn,1,5,2008-07-08,Amazing story,Great story that really draws you in. I am not...,...,"[826, 2095, 8872, 19070, 25172]",56035,2360,160,English,826,2095,8872,19070,25172
128,0553280589,Eva Luna,10.85,Unknown,Unknown,41,5,1997-02-12,The author's use of magic realism will spur yo...,Are you looking to add some vivid color into y...,...,"[403, 1956, 11041, 20237, 15737]",49374,2086,307,English,403,1956,11041,20237,15737
216,0394751019,The Persian Boy,10.85,A25QX3XJJCZ23C,Hannah Lovette,4,5,2009-06-12,Truly Amazing (And this is coming from a reluc...,First off I want to say that I was not initial...,...,"[90, 302, 1497, 3301, 4102]",9292,705,420,English,90,302,1497,3301,4102
489,0451204727,Black Cross,34.00,A3R1MVZG7ZZ295,"Sherrie A., Norman ""SherrieNorman""",1,5,2011-03-26,Black Cross,Black CrossI have always been most interested ...,...,"[107, 278, 1783, 4556, 5356]",12080,678,556,English,107,278,1783,4556,5356
614,0816635277,The Good Woman of Setzuan,10.00,A3429NWUW5IYYY,Reader99,0,1,2011-01-23,Don't know-can't get what I ordered.,"Ordered ""The Good Woman of Setzuan by Brecht"" ...",...,"[405, 1064, 2949, 3843, 2708]",10969,378,112,English,405,1064,2949,3843,2708


In [126]:
import ast

# Convert string representation of list to actual list
merged_df['author'] = merged_df['author'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

merged_df['Primary Author'] = merged_df['author'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

In [127]:
# Display the cleaned author column
print(merged_df[['author', 'Primary Author']].head())

merged_df['Primary Author'] = merged_df['Primary Author'].fillna('Unknown')

merged_df.head()

                                           author   Primary Author
0    [Kilian Plunkett, Mark Millar, Dave Johnson]  Kilian Plunkett
128                              [Isabel Allende]   Isabel Allende
216                                [Mary Renault]     Mary Renault
489                                   [Greg Iles]        Greg Iles
614                [Eric Bentley, Bertolt Brecht]     Eric Bentley


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,...,publisher,author,characters,places,ratingHistogram,ratingsCount,reviewsCount,numPages,language,Primary Author
0,1401201911,Superman: Red Son,11.42,A3BGNNL6VPLFRO,Michael Kleyn,1,5,2008-07-08,Amazing story,Great story that really draws you in. I am not...,...,DC Comics,"[Kilian Plunkett, Mark Millar, Dave Johnson]","['Lana Lang', 'Martha Clark Kent', 'Oliver Que...",NaN,"[826, 2095, 8872, 19070, 25172]",56035,2360,160,English,Kilian Plunkett
128,0553280589,Eva Luna,10.85,Unknown,Unknown,41,5,1997-02-12,The author's use of magic realism will spur yo...,Are you looking to add some vivid color into y...,...,Bantam,[Isabel Allende],"['El Negro', 'Rolf Carlé', 'Huberto Naranjo', ...",['Chile'],"[403, 1956, 11041, 20237, 15737]",49374,2086,307,English,Isabel Allende
216,0394751019,The Persian Boy,10.85,A25QX3XJJCZ23C,Hannah Lovette,4,5,2009-06-12,Truly Amazing (And this is coming from a reluc...,First off I want to say that I was not initial...,...,Vintage,[Mary Renault],"['Darius III', 'Roxana of Bactria', 'Hephaesti...","['Persia', 'Persian Empire']","[90, 302, 1497, 3301, 4102]",9292,705,420,English,Mary Renault
489,0451204727,Black Cross,34.00,A3R1MVZG7ZZ295,"Sherrie A., Norman ""SherrieNorman""",1,5,2011-03-26,Black Cross,Black CrossI have always been most interested ...,...,New American Library,[Greg Iles],"['Jonas Stern', 'Mark McConnell', 'Anna Haas',...",NaN,"[107, 278, 1783, 4556, 5356]",12080,678,556,English,Greg Iles
614,0816635277,The Good Woman of Setzuan,10.00,A3429NWUW5IYYY,Reader99,0,1,2011-01-23,Don't know-can't get what I ordered.,"Ordered ""The Good Woman of Setzuan by Brecht"" ...",...,University of Minnesota Press,"[Eric Bentley, Bertolt Brecht]","['Shui Ta', 'Shen Te', 'Wang']",['China'],"[405, 1064, 2949, 3843, 2708]",10969,378,112,English,Eric Bentley


In [128]:
# Drop the 'author' column
merged_df.drop(columns=['author'], inplace=True)

# Display updated DataFrame
merged_df

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,...,isbn,publisher,characters,places,ratingHistogram,ratingsCount,reviewsCount,numPages,language,Primary Author
0,1401201911,Superman: Red Son,11.42,A3BGNNL6VPLFRO,Michael Kleyn,1,5,2008-07-08,Amazing story,Great story that really draws you in. I am not...,...,1401201911,DC Comics,"['Lana Lang', 'Martha Clark Kent', 'Oliver Que...",NaN,"[826, 2095, 8872, 19070, 25172]",56035,2360,160,English,Kilian Plunkett
128,0553280589,Eva Luna,10.85,Unknown,Unknown,41,5,1997-02-12,The author's use of magic realism will spur yo...,Are you looking to add some vivid color into y...,...,0553280589,Bantam,"['El Negro', 'Rolf Carlé', 'Huberto Naranjo', ...",['Chile'],"[403, 1956, 11041, 20237, 15737]",49374,2086,307,English,Isabel Allende
216,0394751019,The Persian Boy,10.85,A25QX3XJJCZ23C,Hannah Lovette,4,5,2009-06-12,Truly Amazing (And this is coming from a reluc...,First off I want to say that I was not initial...,...,0394751019,Vintage,"['Darius III', 'Roxana of Bactria', 'Hephaesti...","['Persia', 'Persian Empire']","[90, 302, 1497, 3301, 4102]",9292,705,420,English,Mary Renault
489,0451204727,Black Cross,34.00,A3R1MVZG7ZZ295,"Sherrie A., Norman ""SherrieNorman""",1,5,2011-03-26,Black Cross,Black CrossI have always been most interested ...,...,0451204727,New American Library,"['Jonas Stern', 'Mark McConnell', 'Anna Haas',...",NaN,"[107, 278, 1783, 4556, 5356]",12080,678,556,English,Greg Iles
614,0816635277,The Good Woman of Setzuan,10.00,A3429NWUW5IYYY,Reader99,0,1,2011-01-23,Don't know-can't get what I ordered.,"Ordered ""The Good Woman of Setzuan by Brecht"" ...",...,0816635277,University of Minnesota Press,"['Shui Ta', 'Shen Te', 'Wang']",['China'],"[405, 1064, 2949, 3843, 2708]",10969,378,112,English,Eric Bentley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9284,1896944329,Earthcore,10.85,A3LPFHKYNGKDHY,mario,4,4,2007-08-10,Great action book!,If you want to read a great action & adventure...,...,1896944329,Dragon Moon Press,NaN,NaN,"[101, 287, 1335, 2638, 2621]",6982,538,320,Unknown,Scott Sigler
9358,0679729895,Tales of the Unexpected,11.63,A2JBDC1OR1KZGP,Catherine S. Vodrey,29,5,2002-03-17,"Vintage Roald Dahl--wonderful, wonderful!",Roald Dahl is at the peak of his powers in thi...,...,0679729895,Random House Vintage Books,NaN,NaN,"[51, 256, 1596, 3940, 3980]",9823,533,471,English,Roald Dahl
9391,0156032899,Lighthousekeeping,16.95,A319KYEIAZ3SON,Mary Whipple,28,5,2005-04-14,"""My life is a hesitation in time, an opening i...",Jeanette Winterson's magnificently descriptive...,...,0156032899,Mariner Books,NaN,NaN,"[123, 594, 2242, 3317, 2618]",8894,872,232,English,Jeanette Winterson
9413,1591169151,"Ouran High School Host Club, Vol. 1",9.99,ACMK7RRUP9UEW,Haine,2,5,2006-11-09,Hillarious!,I enjoyed this book alot. Hatori-sensei has re...,...,1591169151,VIZ Media,"['Tamaki Suou', 'Mitsukuni Haninozuka', 'Hikar...",NaN,"[3798, 5184, 19221, 38040, 106063]",172306,1731,178,English,Bisco Hatori


In [129]:
merged_df.rename(columns={'Primary Author': 'Author'}, inplace=True)

# Display updated DataFrame
merged_df.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,...,isbn,publisher,characters,places,ratingHistogram,ratingsCount,reviewsCount,numPages,language,Author
0,1401201911,Superman: Red Son,11.42,A3BGNNL6VPLFRO,Michael Kleyn,1,5,2008-07-08,Amazing story,Great story that really draws you in. I am not...,...,1401201911,DC Comics,"['Lana Lang', 'Martha Clark Kent', 'Oliver Que...",NaN,"[826, 2095, 8872, 19070, 25172]",56035,2360,160,English,Kilian Plunkett
128,0553280589,Eva Luna,10.85,Unknown,Unknown,41,5,1997-02-12,The author's use of magic realism will spur yo...,Are you looking to add some vivid color into y...,...,0553280589,Bantam,"['El Negro', 'Rolf Carlé', 'Huberto Naranjo', ...",['Chile'],"[403, 1956, 11041, 20237, 15737]",49374,2086,307,English,Isabel Allende
216,0394751019,The Persian Boy,10.85,A25QX3XJJCZ23C,Hannah Lovette,4,5,2009-06-12,Truly Amazing (And this is coming from a reluc...,First off I want to say that I was not initial...,...,0394751019,Vintage,"['Darius III', 'Roxana of Bactria', 'Hephaesti...","['Persia', 'Persian Empire']","[90, 302, 1497, 3301, 4102]",9292,705,420,English,Mary Renault
489,0451204727,Black Cross,34.00,A3R1MVZG7ZZ295,"Sherrie A., Norman ""SherrieNorman""",1,5,2011-03-26,Black Cross,Black CrossI have always been most interested ...,...,0451204727,New American Library,"['Jonas Stern', 'Mark McConnell', 'Anna Haas',...",NaN,"[107, 278, 1783, 4556, 5356]",12080,678,556,English,Greg Iles
614,0816635277,The Good Woman of Setzuan,10.00,A3429NWUW5IYYY,Reader99,0,1,2011-01-23,Don't know-can't get what I ordered.,"Ordered ""The Good Woman of Setzuan by Brecht"" ...",...,0816635277,University of Minnesota Press,"['Shui Ta', 'Shen Te', 'Wang']",['China'],"[405, 1064, 2949, 3843, 2708]",10969,378,112,English,Eric Bentley


In [103]:
# Reorganizing and cleaning column order for better readability
merged_df.rename(columns={
    'isbn': 'ISBN',  # Ensure ISBN is correctly named
    'title': 'Title',  # Assuming 'title' is the title column
    'author': 'Author',  # Ensure author is correctly named
    'genres': 'Genres',  # Adjust column names to match your list
    'price': 'Price',  # Ensure price is correctly named
    '1_star': '1 Star',  # Adjust column names to match your list
    '2_star': '2 Star',  # Adjust column names to match your list
    '3_star': '3 Star',  # Adjust column names to match your list
    '4_star': '4 Star',  # Adjust column names to match your list
    '5_star': '5 Star',  # Adjust column names to match your list
    'ratingsCount': 'Ratings Count',  # Ensure ratingsCount is correctly named
    'reviewsCount': 'Reviews Count',  # Ensure reviewsCount is correctly named
    'review/helpfulness': 'Review Helpfulness',  # Adjust column names to match your list
    'review/score': 'Review Score',  # Adjust column names to match your list
    'review/summary': 'Review Summary',  # Adjust column names to match your list
    'review/text': 'Review Text',  # Adjust column names to match your list
    'review/time': 'Review Time',  # Adjust column names to match your list
    'description': 'Description',  # Ensure description is correctly named
    'publisher': 'Publisher',  # Ensure publisher is correctly named
    'characters': 'Characters',  # Adjust column names to match your list
    'places': 'Places',  # Adjust column names to match your list
    'numPages': 'Numero Pages',  # Adjust column names to match your list
    'language': 'Language'  # Ensure language is correctly named
}, inplace=True)

# Define the desired order of columns
desired_order = [  # Define the desired order of columns
    'ISBN', 'Title', 'Author', 'Genres', 'Price', '1 Star', '2 Star', '3 Star', '4 Star', '5 Star',
    'Ratings Count', 'Reviews Count', 'Review Helpfulness', 'Review Score', 'Review Summary', 'Review Text',
    'Review Time', 'Description', 'Publisher', 'Characters', 'Places', 'Numero Pages', 'Language'
]

# Reorder columns while keeping only existing ones
merged_df = merged_df[[col for col in desired_order if col in merged_df.columns]]
merged_df.head()

,ISBN,Title,Author,Genres,Price,1 Star,2 Star,3 Star,4 Star,5 Star,...,Review Score,Review Summary,Review Text,Review Time,Description,Publisher,Characters,Places,Numero Pages,Language
0,1401201911,Superman: Red Son,Kilian Plunkett,"['Comics', 'Superheroes', 'Comic Book', 'Graph...",11.42,826,2095,8872,19070,25172,...,5,Amazing story,Great story that really draws you in. I am not...,2008-07-08,Strange visitor from another world who can cha...,DC Comics,"['Lana Lang', 'Martha Clark Kent', 'Oliver Que...",NaN,160,English
128,0553280589,Eva Luna,Isabel Allende,"['Novels', 'Literature', 'Classics', 'Historic...",10.85,403,1956,11041,20237,15737,...,5,The author's use of magic realism will spur yo...,Are you looking to add some vivid color into y...,1997-02-12,Meet New York Times bestselling author Isabel ...,Bantam,"['El Negro', 'Rolf Carlé', 'Huberto Naranjo', ...",['Chile'],307,English
216,0394751019,The Persian Boy,Mary Renault,"['Greece', 'Classics', 'Romance', 'LGBT', 'His...",10.85,90,302,1497,3301,4102,...,5,Truly Amazing (And this is coming from a reluc...,First off I want to say that I was not initial...,2009-06-12,"“It takes skill to depict, as Miss Renault has...",Vintage,"['Darius III', 'Roxana of Bactria', 'Hephaesti...","['Persia', 'Persian Empire']",420,English
489,0451204727,Black Cross,Greg Iles,"['Historical', 'Historical Fiction', 'World Wa...",34.00,107,278,1783,4556,5356,...,5,Black Cross,Black CrossI have always been most interested ...,2011-03-26,It is January 1944 — and as Allied troops prep...,New American Library,"['Jonas Stern', 'Mark McConnell', 'Anna Haas',...",NaN,556,English
614,0816635277,The Good Woman of Setzuan,Eric Bentley,"['Read For School', 'Theatre', 'Drama', 'Ficti...",10.00,405,1064,2949,3843,2708,...,1,Don't know-can't get what I ordered.,"Ordered ""The Good Woman of Setzuan by Brecht"" ...",2011-01-23,"In 1952, Hannah Arendt hailed Bertolt Brecht a...",University of Minnesota Press,"['Shui Ta', 'Shen Te', 'Wang']",['China'],112,English


In [135]:
# Save the merged_df DataFrame to a CSV file
merged_df.to_csv('book_recommender_.csv', index=False)